Data base reading (mongodb) and crate csv files for overall equipment effectiveness analysis

In [2]:
import pandas as pd
import json
import bson

In [3]:
import pymongo
from pymongo import MongoClient
client=MongoClient('localhost' ,27017)

In [4]:
print(client.list_database_names())

['BRIX', 'BRIX1', 'BRIX2', 'admin', 'config', 'ella', 'ella1', 'local']


In [5]:
dblist = client.list_database_names()
if "BRIX" in dblist:
  print("The database exists.")

The database exists.


In [6]:
mydb = client["BRIX"]
mydb

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'BRIX')

In [7]:
print(mydb.list_collection_names())

['events', 'PackML', 'telemetry', 'Procedures']


In [8]:
packmldb = mydb["PackML"]
packmldb 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'BRIX'), 'PackML')

In [9]:
x = packmldb.find_one()
x

{'_id': ObjectId('61a8d62e94cd171694a5f02f'),
 'topic': 'brix/PackML/Egg sampler',
 'payload': {'oldMode': 1,
  'newMode': 1,
  'oldState': 2,
  'newState': 2,
  'oldModeName': 'Production',
  'newModeName': 'Production',
  'oldStateName': 'Stopped',
  'newStateName': 'Stopped',
  'changeTime': '2021-12-02T14:19:15.280Z',
  'durationHr': 0.01942027777777778}}

In [13]:
#df = pd.read_csv('topic.csv', index_col=0)
df = pd.read_csv('topic_Egg sampler_Product sampler.csv')

dataframes_list = []

for va in df['BB']:
    myquery = { "topic": va }
    packml_dataframe=pd.DataFrame.from_dict(list(packmldb .find(myquery)))
    #extract payload colomn
    packml_payload=packml_dataframe.payload.apply(pd.Series)

    concatenated_packml_payload = pd.concat([packml_dataframe, packml_payload], axis="columns")

    # date and time add from change time
    packml_payload [['date', 'time']] =packml_payload ['changeTime'].str.split('T', expand=True)
    packml_changeTime=packml_payload [['date', 'time']]
    #packml_payload [['date', 'time']].head()

    #date and time connect 
    concatenated_packml_payload_Time = pd.concat([concatenated_packml_payload , packml_changeTime], axis=1)
    #concatenated_packml_payload_Time.head()

    #Remove payload
    PackML_Eggsampler_Productsampler=concatenated_packml_payload_Time.drop(columns="payload")
    #PackML_Eggsampler_Productsampler.head()
    
    # append datasets into the list
    dataframes_list.append(PackML_Eggsampler_Productsampler)
#note    
# need to crate : Write each DataFrame to separate CSV 
#PackML_Eggsampler_Productsampler.to_csv("Product sampler_Product sampler transporter 05.csv")

In [14]:
pd.DataFrame(dataframes_list[0]).to_csv("new_hcp.csv")
dataframes_list[0].head()

,_id,topic,oldMode,newMode,oldState,newState,oldModeName,newModeName,oldStateName,newStateName,changeTime,durationHr,date,time
0,620b60de32d6c511bc657efe,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:16:08.834Z,15.975613,2022-02-15,08:16:08.834Z
1,620b614a32d6c511bc6583e7,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:17:57.338Z,0.012997,2022-02-15,08:17:57.338Z
2,620b64ecee0f66110041e8b9,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:33:27.945Z,0.211141,2022-02-15,08:33:27.945Z
3,620b659cee0f66110041f04f,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:36:24.175Z,0.029459,2022-02-15,08:36:24.175Z
4,620b65adee0f66110041f14c,brix/PackML/Egg sampler/Product sampler,3,3,2,15,Manual,Manual,Stopped,Resetting,2022-02-15T08:36:40.534Z,0.004542,2022-02-15,08:36:40.534Z


In [15]:
# export a list of df to separate csv files 
for j, frame in enumerate(dataframes_list):
    frame.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/OEE calculation/Product sampler data/DB csv/db{j}.csv")

    
    #print(dataframes_list[j]['topic'].values[0])

In [16]:
#Import multiple CSV files into pandas and concatenate into one DataFrame
import os
import glob
path = "C:/Users/HLQ/Documents/jupyter/EGSS/OEE calculation/Product sampler data/DB csv"
allFiles = glob.glob(os.path.join(path,"*.csv"))


dataframes_list_1 = []
for file_ in allFiles:
    df3 = pd.read_csv(file_,index_col=0)

    dataframes_list_1.append(df3)

In [17]:
dataframes_list_1[0].head()

,_id,topic,oldMode,newMode,oldState,newState,oldModeName,newModeName,oldStateName,newStateName,changeTime,durationHr,date,time
0,620b60de32d6c511bc657efe,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:16:08.834Z,15.975613,2022-02-15,08:16:08.834Z
1,620b614a32d6c511bc6583e7,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:17:57.338Z,0.012997,2022-02-15,08:17:57.338Z
2,620b64ecee0f66110041e8b9,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:33:27.945Z,0.211141,2022-02-15,08:33:27.945Z
3,620b659cee0f66110041f04f,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:36:24.175Z,0.029459,2022-02-15,08:36:24.175Z
4,620b65adee0f66110041f14c,brix/PackML/Egg sampler/Product sampler,3,3,2,15,Manual,Manual,Stopped,Resetting,2022-02-15T08:36:40.534Z,0.004542,2022-02-15,08:36:40.534Z


In [18]:
brix1packml = dataframes_list_1[0]
brix1packml.head()
#brix1packml.info()

,_id,topic,oldMode,newMode,oldState,newState,oldModeName,newModeName,oldStateName,newStateName,changeTime,durationHr,date,time
0,620b60de32d6c511bc657efe,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:16:08.834Z,15.975613,2022-02-15,08:16:08.834Z
1,620b614a32d6c511bc6583e7,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:17:57.338Z,0.012997,2022-02-15,08:17:57.338Z
2,620b64ecee0f66110041e8b9,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:33:27.945Z,0.211141,2022-02-15,08:33:27.945Z
3,620b659cee0f66110041f04f,brix/PackML/Egg sampler/Product sampler,1,3,2,2,Production,Manual,Stopped,Stopped,2022-02-15T08:36:24.175Z,0.029459,2022-02-15,08:36:24.175Z
4,620b65adee0f66110041f14c,brix/PackML/Egg sampler/Product sampler,3,3,2,15,Manual,Manual,Stopped,Resetting,2022-02-15T08:36:40.534Z,0.004542,2022-02-15,08:36:40.534Z


In [20]:
brix1packml["oldStateName"].value_counts()

Execute      10201
Starting      9895
Stopped       7614
Resetting     4392
Idle          4014
Aborted       3727
Stopping      3350
Aborting      3194
Clearing      2878
Held           986
Holding        969
Unholding      466
Name: oldStateName, dtype: int64

##OEE

Execute      
Starting     
Stopped      
Resetting    
Aborted     
Idle         
Aborting     
Stopping     
Clearing     
Held          
Holding       
Unholding

***Availability***<br>
**Availablity = T𝑖𝑚𝑒 𝑚𝑎𝑐ℎ𝑖𝑛𝑒 𝑖𝑠 𝑃𝑟𝑜𝑑𝑢𝑐𝑖𝑛𝑔 𝑃𝑟𝑜𝑑𝑢𝑐𝑡 (𝐸𝑥𝑒𝑐𝑢𝑡𝑒 + 𝐻𝑜𝑙𝑑𝑖𝑛𝑔 + 𝑆𝑢𝑠𝑝𝑒𝑛𝑑𝑖𝑛𝑔) /(T𝑖𝑚𝑒 𝑚𝑎𝑐ℎ𝑖𝑛𝑒 𝑖𝑠 𝑃𝑟𝑜𝑑𝑢𝑐𝑖𝑛𝑔 𝑃𝑟𝑜𝑑𝑢𝑐𝑡 + 𝑇𝑖𝑚𝑒     𝑤ℎ𝑒𝑛 𝑚𝑎𝑐ℎ𝑖𝑛𝑒 𝑖𝑠 𝑖𝑛 𝑓𝑎𝑙𝑖𝑢𝑟𝑒(𝐻𝑒𝑙𝑑+𝑆𝑡𝑜𝑝𝑝𝑒𝑑+𝐴𝑏𝑜𝑟𝑡𝑒𝑑))**

Availability is a percentage of the time the equipment/line  spent producing products (in the PackML Execute State, Suspending State and Holding State) 

**Performance**<br>
**Performance= Total quantity produced at this machine/(𝑇𝑖𝑚𝑒 𝑚𝑎𝑐ℎ𝑖𝑛𝑒 𝑖𝑠 𝑝𝑟𝑜𝑑𝑢𝑐𝑖𝑛𝑔 𝑃𝑟𝑜𝑑𝑢𝑐𝑡 (𝐸𝑥𝑒𝑐𝑢𝑡𝑒) ∗ 𝑀𝑎𝑥𝑖𝑚𝑢𝑚 𝑀𝑎𝑐ℎ𝑖𝑛𝑒 𝑆𝑝𝑒𝑒d)**




In [21]:
dataframes_list1=[]
for brix1packml in dataframes_list_1:
    #Availability
    #filter [newStateName']=="Execute") or ['newStateName']=="Holding"
    #time_machine_producing_product
    a=(brix1packml['oldStateName']=="Execute") | (brix1packml['oldStateName']=="Holding")
    Execute_Holding_df=brix1packml[a]
    
    #time_machine_faliure
    # filter 'newStateName']=="Held" or  "Stopped" or "Aborted"
    a1=(brix1packml['oldStateName']=="Held") | (brix1packml['oldStateName']=="Stopped") | (brix1packml['oldStateName']=="Aborted")
    Held_Stopped_Aborted_df=brix1packml[a1]

    #Performance
    E=(brix1packml['oldStateName']=="Execute")
    Execute_df=brix1packml[E]
    
    H=(brix1packml['oldStateName']=="Holding")
    Holding_df=brix1packml[H]

    He=(brix1packml['oldStateName']=="Held")
    Held_df=brix1packml[He]

    S=(brix1packml['oldStateName']=="Stopped")
    Stopped_df=brix1packml[S]

    Ab=(brix1packml['oldStateName']=="Aborted")
    Aborted_df=brix1packml[Ab]
    
    Holding_df['date'] = pd.to_datetime(Holding_df['date'])
    delily_summary_time_machine_producing_product_Holding = Holding_df.resample('d', on='date').sum()
    delily_summary_time_machine_producing_product_Holding1=delily_summary_time_machine_producing_product_Holding.rename({'durationHr':'HoldingHr'}, axis=1)
    
    Held_df['date'] = pd.to_datetime(Held_df['date'])
    delily_summary_time_machine_producing_product_Held = Held_df.resample('d', on='date').sum()
    delily_summary_time_machine_producing_product_Held1=delily_summary_time_machine_producing_product_Held.rename({'durationHr':'HeldHr'}, axis=1)
   
    Stopped_df['date'] = pd.to_datetime(Stopped_df['date'])
    delily_summary_time_machine_producing_product_Stopped= Stopped_df.resample('d', on='date').sum()
    delily_summary_time_machine_producing_product_Stopped1=delily_summary_time_machine_producing_product_Stopped.rename({'durationHr':'StoppedHr'}, axis=1)
    
    Aborted_df['date'] = pd.to_datetime(Aborted_df['date'])
    delily_summary_time_machine_producing_product_Aborted = Aborted_df.resample('d', on='date').sum()
    delily_summary_time_machine_producing_product_Aborted1=delily_summary_time_machine_producing_product_Aborted.rename({'durationHr':'AbortedHr'}, axis=1)
    
    #delily_summary_time_machine_producing_product_Execute
    Execute_df['date'] = pd.to_datetime(Execute_df['date'])
    delily_summary_time_machine_producing_product_Execute = Execute_df.resample('d', on='date').sum()
    
    #rename 'durationHr':'ExecuteHr'
    delily_summary_time_machine_producing_product_Execute1=delily_summary_time_machine_producing_product_Execute.rename({'durationHr':'ExecuteHr'}, axis=1)
    
    #delily_summary_time_machine_producing_product
    Execute_Holding_df['date'] = pd.to_datetime(Execute_Holding_df['date'])
    delily_summary_time_machine_producing_product = Execute_Holding_df.resample('d', on='date').sum()
    delily_summary_time_machine_producing_product1=delily_summary_time_machine_producing_product.rename({'durationHr':'P_productHr'}, axis=1)
    
    #delily_time_machine_faliure
    Held_Stopped_Aborted_df['date'] = pd.to_datetime(Held_Stopped_Aborted_df['date'])
    delily_time_machine_faliure = Held_Stopped_Aborted_df.resample('d', on='date').sum()
    
    #rename durationHr
    delily_time_machine_faliure1=delily_time_machine_faliure.rename({'durationHr':'faliureHr'}, axis=1)

    # new for add packml state
    extracted_col11 = delily_summary_time_machine_producing_product_Holding1["HoldingHr"]
    Availability_data1 = delily_summary_time_machine_producing_product_Held1.join(extracted_col11)

    extracted_col12 =delily_summary_time_machine_producing_product_Stopped1["StoppedHr"]
    Availability_data2 = Availability_data1.join(extracted_col12)

    extracted_col13 =delily_summary_time_machine_producing_product_Aborted1["AbortedHr"]
    Availability_data3 = Availability_data2.join(extracted_col13)

    extracted_col14 = delily_summary_time_machine_producing_product_Execute1["ExecuteHr"]
    Availability_data4 = Availability_data3.join(extracted_col14)

    extracted_col15 = delily_time_machine_faliure1["faliureHr"]
    Availability_data5 = Availability_data4.join(extracted_col15)

    extracted_col16 = delily_summary_time_machine_producing_product1["P_productHr"]
    Availability_Performance_data = Availability_data5.join(extracted_col16)
    Availability_Performance_data.head()
    
    Availability_Performance_data = Availability_Performance_data.reset_index()
    
    # calculate and add Availability 
    Availability_Performance_data['Availability']=Availability_Performance_data['P_productHr']/(Availability_Performance_data['P_productHr']+Availability_Performance_data['faliureHr'])
   
    # drop colomns
    Availability_Performance_data_copy = Availability_Performance_data.copy()
    Availability_Performance_data_copy=Availability_Performance_data_copy.drop(columns=['oldMode', 'newMode', 'oldState', 'newState'])
    
    #filter date between 2022_04_15 to 2022_10_13
    Availability_Performance_2022_0415_1013 = Availability_Performance_data_copy.loc[(Availability_Performance_data_copy["date"] >= '2022-04-15')
                     & (Availability_Performance_data_copy["date"] < '2022-10-14')]
    Availability_Performance_2022_0415_1013.head()
    
    #Note
    #need to crate new csv files 
    #Availability_Performance_2022_0415_1013.to_csv("C:/Users/HLQ/Documents/jupyter/Eggs sampler/BRIX OEE_newdata/OEE data/Availability_Performance_data/Availability_Performance_2022_0415_1013.csv")

    
    dataframes_list1.append(Availability_Performance_2022_0415_1013)
#Execute_Holding_df.head() 

C:\Temp\ipykernel_12120\3639848308.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Holding_df['date'] = pd.to_datetime(Holding_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Held_df['date'] = pd.to_datetime(Held_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

C:\Temp\ipykernel_12120\3639848308.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Holding_df['date'] = pd.to_datetime(Holding_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Held_df['date'] = pd.to_datetime(Held_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

C:\Temp\ipykernel_12120\3639848308.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Held_df['date'] = pd.to_datetime(Held_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Stopped_df['date'] = pd.to_datetime(Stopped_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

C:\Temp\ipykernel_12120\3639848308.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Holding_df['date'] = pd.to_datetime(Holding_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Held_df['date'] = pd.to_datetime(Held_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

C:\Temp\ipykernel_12120\3639848308.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Holding_df['date'] = pd.to_datetime(Holding_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Held_df['date'] = pd.to_datetime(Held_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

C:\Temp\ipykernel_12120\3639848308.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Held_df['date'] = pd.to_datetime(Held_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Stopped_df['date'] = pd.to_datetime(Stopped_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

C:\Temp\ipykernel_12120\3639848308.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Execute_df['date'] = pd.to_datetime(Execute_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Execute_Holding_df['date'] = pd.to_datetime(Execute_Holding_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

C:\Temp\ipykernel_12120\3639848308.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Execute_df['date'] = pd.to_datetime(Execute_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Execute_Holding_df['date'] = pd.to_datetime(Execute_Holding_df['date'])
C:\Temp\ipykernel_12120\3639848308.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [22]:
dataframes_list1[0]

,date,HeldHr,HoldingHr,StoppedHr,AbortedHr,ExecuteHr,faliureHr,P_productHr,Availability
59,2022-04-15,0.002567,0.000606,17.631387,0.014661,0.250836,17.648614,0.251442,0.014047
60,2022-04-16,0.000000,0.000000,24.000000,0.000000,0.000000,24.000000,0.000000,0.000000
61,2022-04-17,0.000000,0.000000,24.000003,0.000000,0.000000,24.000003,0.000000,0.000000
62,2022-04-18,0.000000,0.000000,24.000003,0.000000,0.000000,24.000003,0.000000,0.000000
63,2022-04-19,0.578294,0.011489,21.540483,0.273593,10.895875,22.392370,10.907364,0.327551
...,...,...,...,...,...,...,...,...,...
236,2022-10-09,0.000000,0.000000,24.000003,0.000000,0.000000,24.000003,0.000000,0.000000
237,2022-10-10,0.000000,0.000000,22.565478,0.004093,0.000000,22.569571,0.000000,0.000000
238,2022-10-11,0.000000,0.000000,0.275194,0.039050,1.793619,0.314243,1.793619,0.850919
239,2022-10-12,0.000000,0.000000,18.540948,0.378941,0.214253,18.919889,0.214253,0.011197


In [23]:
dataframes_list1[0].dtypes

date            datetime64[ns]
HeldHr                 float64
HoldingHr              float64
StoppedHr              float64
AbortedHr              float64
ExecuteHr              float64
faliureHr              float64
P_productHr            float64
Availability           float64
dtype: object

In [24]:
# Egg production data csv load
Egg_production = pd.read_csv("Egg production.csv", index_col=0)
Egg_production

,date,trays,Egg_production
79,2022-04-15,4,560
29,2022-04-19,64,8960
27,2022-04-20,74,10360
89,2022-04-21,1,140
54,2022-04-22,17,2380
...,...,...,...
3,2022-10-04,293,41020
41,2022-10-06,27,3780
42,2022-10-07,26,3640
83,2022-10-12,3,420


In [25]:
Egg_production.dtypes

date              object
trays              int64
Egg_production     int64
dtype: object

In [26]:
# calulate OEE with Egg production
#Availability_Performance_2022_0415_1013 =dataframes_list1

Maximum_Machine_speed= 7000
dataframes_list2=[]
for AP in dataframes_list1:
   
    #merge Availabulity Execute and production
    df2=AP
    df1=Egg_production
    df1['date'] = pd.to_datetime(df1['date'])
    AP_df = df1.merge(df2, on='date')
###    
#Performance
#Performance= Total quantity produced at this machine/(𝑇𝑖𝑚𝑒 𝑚𝑎𝑐ℎ𝑖𝑛𝑒 𝑖𝑠 𝑝𝑟𝑜𝑑𝑢𝑐𝑖𝑛𝑔 𝑃𝑟𝑜𝑑𝑢𝑐𝑡(𝐸𝑥𝑒𝑐𝑢𝑡𝑒) ∗ 𝑀𝑎𝑥𝑖𝑚𝑢𝑚 𝑀𝑎𝑐ℎ𝑖𝑛𝑒 𝑆𝑝𝑒𝑒d)
#𝑀𝑎𝑥𝑖𝑚𝑢𝑚 𝑀𝑎𝑐ℎ𝑖𝑛𝑒 𝑆𝑝𝑒𝑒d= 7000 Egg/Hr
#Running 𝑀𝑎𝑐ℎ𝑖𝑛𝑒 𝑆𝑝𝑒𝑒d = 4000 Egg/Hr

    AP_df['Performance']=AP_df['Egg_production']/(AP_df['ExecuteHr']*Maximum_Machine_speed)
    AP_df['OEE']=AP_df['Availability']*AP_df['Performance']
   
    dataframes_list2.append(AP_df)

In [27]:
# export a list of df to separate csv files 
for j1, frame1 in enumerate(dataframes_list2):
      #frame.to_csv(f"dff{j+1}.csv")
    #frame.to_csv('C:/Users/HLQ/Documents/jupyter/Eggs sampler/OEE all section reading/DB read/db'+str(j)+'.csv')
    frame1.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/OEE calculation/Product sampler data/OEE csv/OEE{j1}.csv")
    
    #print(dftopic_list[j]['topic'].values[0])

In [28]:
dataframes_list2[0]

,date,trays,Egg_production,HeldHr,HoldingHr,StoppedHr,AbortedHr,ExecuteHr,faliureHr,P_productHr,Availability,Performance,OEE
0,2022-04-15,4,560,0.002567,0.000606,17.631387,0.014661,0.250836,17.648614,0.251442,0.014047,0.318933,0.004480
1,2022-04-19,64,8960,0.578294,0.011489,21.540483,0.273593,10.895875,22.392370,10.907364,0.327551,0.117476,0.038479
2,2022-04-20,74,10360,0.736500,0.011497,8.997572,0.522766,7.936753,10.256838,7.948250,0.436595,0.186474,0.081414
3,2022-04-21,1,140,0.000000,0.000000,13.287334,0.045575,0.030761,13.332909,0.030761,0.002302,0.650172,0.001497
4,2022-04-22,17,2380,0.015117,0.002861,0.753977,0.430045,1.224011,1.199139,1.226872,0.505716,0.277775,0.140475
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2022-10-04,293,41020,0.038522,0.001211,1.042894,0.006707,14.787217,1.088123,14.788428,0.931463,0.396288,0.369128
86,2022-10-06,27,3780,0.497742,0.004231,1.554899,0.713428,1.550053,2.766069,1.554283,0.359758,0.348375,0.125331
87,2022-10-07,26,3640,0.293019,0.003028,17.414926,2.444098,1.263106,20.152043,1.266133,0.059115,0.411684,0.024337
88,2022-10-12,3,420,0.000000,0.000000,18.540948,0.378941,0.214253,18.919889,0.214253,0.011197,0.280043,0.003136
